In [2]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 267.7/267.7 kB 2.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from nltk.corpus import stopwords
import nltk
nltk.download('popular')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_d

In [5]:
from nltk import bigrams, ngrams

In [9]:
with open('text.txt', 'r',encoding='utf-8') as f:
    text_sample = f.read()

In [10]:
from collections import Counter
import matplotlib.pyplot as plt

def BoW(text, n=1, clean=False):
    tokens = word_tokenize(text) # tokenize text
    
    if clean:
        tokens = clean_tokens(tokens)

    if n > 1: # optionally use bigrams
        tokens = ngrams(tokens, n)
    
    token_frequency = Counter(tokens) # get Counter (like a dictionary) of the frequency of each token
    return token_frequency


def BoW_hist(token_frequency, n_top=10):
    top_tokens = token_frequency.most_common(n_top)

    w, f = [[i for i,j in top_tokens], [j for i,j in top_tokens]]
    
    fig, ax = plt.subplots(figsize=(6 * n_top/10,4))
    ax.bar(range(n_top), f)
    ax.set_xticks(range(n_top))
    ax.set_xticklabels(w, rotation=45, ha='right')
    
    return fig, ax


def clean_tokens(tokens):
    sw = stopwords.words('english') # nltk's collection of English-language stopwords
    punctuation = [',', '.', ';', '“', '”', '’', '‘', '!', '?', '\'\'', '``', '--'] # non-exhaustive list of punctuation commonly found in the three books of interest
    tokens = [s.lower() for s in tokens]
    tokens = [s for s in tokens if s not in punctuation and s not in sw] # make all tokens lowercase
    return tokens

In [58]:
def vectorizer(documents):
    dictionaries = []
    for i in documents:
        dictionaries.append(dict(BoW(i)))
    #print(dictionaries[0])
    if(len(dictionaries)>1):
        keys = [list(dictionaries[i].keys())+list(dictionaries[i+1].keys()) for i in range(len(dictionaries)-1)]
    else:
        keys = list(dictionaries[0].keys())
    vectors = []
    for i in dictionaries:
        print(type(i))
        vectors.append([i.get(keys[0][j],0) for j in range(len(keys[0]))])
    return vectors

